# La Estructura del Estado

> ...

Tipologia:
- Direcciones
- Institutos
- ...

In [53]:
import requests
from rich import print
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import duckdb
import requests
from io import BytesIO
import pymupdf4llm
import  pymupdf
import fs
import collections

## Compilationem

> Download, store, and convert from pdf to markdown (text).

In [2]:
institutionalClassifierURL  = "https://www.digepres.gob.do/wp-content/uploads/2024/08/Clasificador-Institucional.pdf"
mem_fs = fs.open_fs('mem://')
mem_fs.makedirs('datasets')

In [39]:
## Download and write the data to an in-memory file system.
response = requests.get(institutionalClassifierURL)
if response.status_code == 200:
    with mem_fs.open("datasets/Clasificador-Institucional.pdf", 'wb') as handler:
        handler.write(response.content)
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

In [ ]:
## Convert the PDF data to Markdown text using PyMuPDF and pymupdf4llm.
with mem_fs.open("datasets/Clasificador-Institucional.pdf", "rb") as handler:
        in_memory_file = BytesIO(handler.read())
        doc  = pymupdf.open(stream  = in_memory_file, filetype="pdf")

text  =  pymupdf4llm.to_markdown(doc)

with mem_fs.open("datasets/Clasificador-Institucional.md", "w") as handler:
        handler.write(text)

## Processus 

> Parse the data, and generate a csv.

In [48]:
with mem_fs.open("datasets/Clasificador-Institucional.md", "r") as handler:
        lines  = handler.readlines()

In [57]:
## Which lines constitute a 'state organization'?
## We have checked the PDF, and the lines that contain twelve '|' are the organizations I want to parse.
print(collections.Counter([line.count("|") for line in lines]))


Counter({0: 294, 12: 2354, 10: 289, 4: 9})

In [60]:
## Parsing
filtered_lines = [line for line in lines if line.count('|') == 12]
filtered_lines = filtered_lines[2:-1]

print(filtered_lines)

[
    '|SECTOR|SUBSECTOR|AREA|||PODERES|ENTIDAD|CAPÍTULO|SUBCAPÍTULO|UNID. EJEC.||\n',
    '|1||||||||||SECTOR PÚBLICO|\n',
    '|1|1|||||||||SECTOR PÚBLICO NO FINANCIERO|\n',
    '|1|1|1||||||||GOBIERNO GENERAL|\n',
    '|1|1|1|1|||||||GOBIERNO CENTRAL|\n',
    '|1|1|1|1|1||||||ADMINISTRACIÓN GENERAL|\n',
    '|1|1|1|1|1|1|||||Poder Legislativo|\n',
    '|1|1|1|1|1|1|1||||Cámara de Senadores|\n'
]

In [77]:
root  = '/home/dvictoriano/Code/Laboratorium'
ds =  pd.read_excel(f'{root}/datasets/CatalogoOrganismos.xlsx')

In [104]:
import fs
import fs.copy

mem_fs = fs.open_fs('mem://')


In [107]:
output  = !curl 'https://map.gob.do/COEDOM/Home/Details/191?Ruta=2' --compressed -H 'User-Agent: Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:131.0) Gecko/20100101 Firefox/131.0' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/png,image/svg+xml,*/*;q=0.8' -H 'Accept-Language: en-US,en;q=0.5' -H 'Accept-Encoding: gzip, deflate, br, zstd' -H 'Referer: https://map.gob.do/COEDOM/Home/SearchAZ?page=1' -H 'Connection: keep-alive' -H 'Cookie: _ga_T38J33J49S=GS1.1.1729788674.2.0.1729788689.45.0.0; _ga=GA1.3.1643063522.1729627237; _ga_TBZ2PDZ7M7=GS1.1.1729788675.2.0.1729788689.46.0.0; __cf_bm=TKlT3afSNLGMqYmU051tYciVVmYUZUKPRaIMIQjgR2E-1729794424-1.0.1.1-.Ky_ACjgxlDZHbhjVGKul151Ji3hLI9nQvgqJRgCQcXuYvd5Au6E6FTJ18cZtnJBMsk6hWRx7B5pAuCcTtrNyA; __bs_id=GA1.1.1643063522.1729627237; _gid=GA1.3.668792708.1729788676; __RequestVerificationToken_L0NPRURPTQ2=nqWoUJzOlh51fbh78Jlaqp0q1ozM-VruFMcDjB-gFfEdA4DiauiWIAyDfX6xz1nAHwvjgDQ3w_2dhSvf8vG3uLRPu2NablZz22RbKZDAt101; cf_clearance=EbtMgQ2CbIov1FGosUljWFkJs0lvO01UuhXBEUMGo2k-1729794292-1.2.1.1-8E7qxuDeEAsH36aUDPgOvx4idGFK3lAWRwlCrk0eHtt8OoTHB5GLZx_9lMe0kYZHGRH9WQSzyOjkwi3uFKKHnoTbEDzVoWF7cSp.OrMfrZvg1DWyyN0RyaSiOtu0ejf1m48d6g8CYhHyXSNhmEH_mFYVwbhsB04DPZoRgFaWS4gmJGQDpc8CIfNJ0H_10Yg1Wr.KxtpJ8TVqh9wXX.4tjkrpj4.HoD_8LOtw.IjTh3Z1WvhtmRpqDsUVvGSjCcp7I1hsMehxG4DAv5AytzdvaBgFoyoVTOS5rDE5FhF6HeBuUkjx3p1z_ggU._tj2b_emZ9XtL98DdJc_TdyoMfHuw' -H 'Upgrade-Insecure-Requests: 1' -H 'Sec-Fetch-Dest: document' -H 'Sec-Fetch-Mode: navigate' -H 'Sec-Fetch-Site: same-origin' -H 'Sec-Fetch-User: ?1' -H 'Priority: u=0, i' -H 'Pragma: no-cache' -H 'Cache-Control: no-cache'

In [62]:
ds = [pd.read_html(StringIO(page))[0] for page in pages]

dsorganismos =  pd.concat(ds)
dsorganismos = dsorganismos[~dsorganismos.iloc[:, 0].str.contains('Siguiente', na=False)]
dsorganismos = dsorganismos[~dsorganismos.iloc[:, 0].str.contains('Primero', na=False)]

columns_to_remove  = ['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5']
dsorganismos = dsorganismos.drop(columns=columns_to_remove)


In [111]:
from autoscraper import AutoScraper

In [113]:
url = 'https://finance.yahoo.com/quote/AAPL/'

wanted_list = ["230.64"]

scraper = AutoScraper()

# Here we can also pass html content via the html parameter instead of the url (html=html_content)
result = scraper.build(url, wanted_list)
print(result)

['230.64', '17.70', '56,600.00', '25.250', '12.70', '45.85', '1.3250', '7.30', '111.00', '76.20', '40.60']

In [118]:
duckdb.sql("""
    SELECT *
    FROM dsorganismos
    where organismo like '%Reforma%'
    """)

┌────────────────────────────────────────────────────────────┬───────────────────────────────────────────┬─────────────┐
│                         Organismo                          │                 Tipología                 │   Sector    │
│                          varchar                           │                  varchar                  │   varchar   │
├────────────────────────────────────────────────────────────┼───────────────────────────────────────────┼─────────────┤
│ Comisión Permanente para la Reforma y Modernización de l…  │ Órgano Colegiado con Estructura Operativa │ Defensa     │
│ Fondo Patrimonial de las Empresas Reformadas (FONPER)      │ Organismo Descentralizado Funcionalmente  │ Presidencia │
└────────────────────────────────────────────────────────────┴───────────────────────────────────────────┴─────────────┘

## Referencias

- [Organismos del Estado Dominicano](https://www.sismap.gob.do/Central/Home/About)

- [Listado de Organismos](https://www.sismap.gob.do/GestionPublica/Organismos)

- [Informacin General de RD](https://www.presidencia.gob.do/acerca-de-rd/informacion-general)

- [Miguel Collado: RD es el cuarto país de América Latina con mayor porcentaje de empleados públicos](https://www.diariolibre.com/economia/macroeconomia/2024/06/20/cuanto-gasta-el-gobierno-en-la-nomina-publica/2761319)

- [Consulta de Organismos](https://map.gob.do/COEDOM/Home/Search)

- https://www.sismap.gob.do/Municipal/

- https://sismap.gob.do/

- [Clasificación de los Organismos de la Administración del Estado Dominicano](https://map.gob.do/COEDOM/Home/Clasificacion)

- [Clasificadores Presupuestarios](https://www.digepres.gob.do/publicaciones/clasificadores-presupuestarios/)